In [1]:
%run init.ipynb

matchzoo version 1.1.1
`ranking_task` initialized with metrics [normalized_discounted_cumulative_gain@3(0.0), normalized_discounted_cumulative_gain@5(0.0), mean_average_precision(0.0)]
data loading ...
/Users/yyang/.matchzoo/datasets/wiki_qa/WikiQACorpus
/Users/yyang/.matchzoo/datasets/wiki_qa/WikiQACorpus
/Users/yyang/.matchzoo/datasets/wiki_qa/WikiQACorpus
data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`


In [2]:
preprocessor = mz.models.Bert.get_default_preprocessor()

In [3]:
train_pack_processed = preprocessor.transform(train_pack_raw)
dev_pack_processed = preprocessor.transform(dev_pack_raw)
test_pack_processed = preprocessor.transform(test_pack_raw)

Processing length_right with len: 100%|██████████| 2300/2300 [00:00<00:00, 757391.79it/s]


In [4]:
trainset = mz.dataloader.Dataset(
    data_pack=train_pack_processed,
    mode='pair',
    num_dup=2,
    num_neg=1
    #resample=True,
    #sort=False,
    #batch_size=20,
)
testset = mz.dataloader.Dataset(
    data_pack=test_pack_processed
    #batch_size=20,
)

In [5]:
padding_callback = mz.models.Bert.get_default_padding_callback()
trainloader = mz.dataloader.DataLoader(
    dataset=trainset,
    stage='train',
    callback=padding_callback
)
testloader = mz.dataloader.DataLoader(
    dataset=testset,
    stage='dev',
    callback=padding_callback
)

In [6]:
model = mz.models.Bert()

model.params['task'] = ranking_task
model.params['mode'] = 'bert-base-uncased'
model.params['dropout_rate'] = 0.2

model.build()

print(model)
print('Trainable params: ', sum(p.numel() for p in model.parameters() if p.requires_grad))

Bert(
  (bert): BertModule(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): BertLayerNorm()
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bias=True)
                (LayerNorm): BertLayerNorm()
                (dr

In [7]:
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 5e-5},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

from pytorch_transformers import AdamW, WarmupLinearSchedule

optimizer = AdamW(optimizer_grouped_parameters, lr=5e-5, betas=(0.9, 0.98), eps=1e-8)
scheduler = WarmupLinearSchedule(optimizer, warmup_steps=6, t_total=-1)

trainer = mz.trainers.Trainer(
    model=model,
    optimizer=optimizer,
    scheduler=scheduler,
    trainloader=trainloader,
    validloader=testloader,
    validate_interval=None,
    epochs=5
)

In [12]:
#rankhingeloss
trainer.run()

/Users/yyang/opt/anaconda3/envs/conda_env/lib/python3.7/site-packages/matchzoo_py-1.1.1-py3.7.egg/matchzoo/data_pack/data_pack.py:151: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x[key] = np.array(val)
/Users/yyang/opt/anaconda3/envs/conda_env/lib/python3.7/site-packages/pytorch_transformers/optimization.py:166: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  /tmp/pip-req-build-rc66hrpz/torch/csrc/utils/python_arg_parser.cpp:766.)
  exp_avg.mul_(beta1).add_(1.0 - beta1, grad)


[Iter-64 Loss-0.922]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.4165 - normalized_discounted_cumulative_gain@5(0.0): 0.4835 - mean_average_precision(0.0): 0.4372



[Iter-128 Loss-0.731]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.7061 - normalized_discounted_cumulative_gain@5(0.0): 0.7441 - mean_average_precision(0.0): 0.7035



[Iter-192 Loss-0.392]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.7966 - normalized_discounted_cumulative_gain@5(0.0): 0.8174 - mean_average_precision(0.0): 0.7775



[Iter-256 Loss-0.202]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.811 - normalized_discounted_cumulative_gain@5(0.0): 0.841 - mean_average_precision(0.0): 0.8022



...

[Iter-384 Loss-0.049]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.8032 - normalized_discounted_cumulative_gain@5(0.0): 0.8285 - mean_average_precision(0.0): 0.7865



[Iter-448 Loss-0.037]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.8114 - normalized_discounted_cumulative_gain@5(0.0): 0.8324 - mean_average_precision(0.0): 0.7966



[Iter-512 Loss-0.037]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.8198 - normalized_discounted_cumulative_gain@5(0.0): 0.8412 - mean_average_precision(0.0): 0.8053



[Iter-576 Loss-0.036]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.8104 - normalized_discounted_cumulative_gain@5(0.0): 0.8352 - mean_average_precision(0.0): 0.7935



[Iter-640 Loss-0.034]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.8145 - normalized_discounted_cumulative_gain@5(0.0): 0.8347 - mean_average_precision(0.0): 0.7974

Cost time: 25511.687927007675s


In [21]:
#rankcrossentropy
trainer.run()

/Users/yyang/opt/anaconda3/envs/conda_env/lib/python3.7/site-packages/matchzoo_py-1.1.1-py3.7.egg/matchzoo/data_pack/data_pack.py:151: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x[key] = np.array(val)


[Iter-64 Loss-0.706]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.2831 - normalized_discounted_cumulative_gain@5(0.0): 0.3655 - mean_average_precision(0.0): 0.3289



[Iter-128 Loss-0.646]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6661 - normalized_discounted_cumulative_gain@5(0.0): 0.7201 - mean_average_precision(0.0): 0.6752



[Iter-192 Loss-0.404]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.7638 - normalized_discounted_cumulative_gain@5(0.0): 0.8003 - mean_average_precision(0.0): 0.7516



[Iter-256 Loss-0.183]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.8103 - normalized_discounted_cumulative_gain@5(0.0): 0.8381 - mean_average_precision(0.0): 0.802



[Iter-320 Loss-0.069]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.8049 - normalized_discounted_cumulative_gain@5(0.0): 0.8344 - mean_average_precision(0.0): 0.7978

Cost time: 12142.689332008362s


In [16]:
print(model)

Bert(
  (bert): BertModule(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): BertLayerNorm()
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bias=True)
                (LayerNorm): BertLayerNorm()
                (dr